In [1]:
import sys
sys.path.append('..')

In [2]:
from data.util import *

In [14]:
dow_johnes = load_data_from_folder(DATA_FOLDERS[0])
ftse_100 = load_data_from_folder(DATA_FOLDERS[1])
nikkei_225 = load_data_from_folder(DATA_FOLDERS[2])
sp_500 = load_data_from_folder(DATA_FOLDERS[3])
EGX_30 = load_data_from_folder(DATA_FOLDERS[4])

dfs = [
    dow_johnes,
    ftse_100,
    nikkei_225,
    sp_500,
    EGX_30]

In [16]:
for df in dfs:
    df = cast_columns_to_double(df)
    df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- Open: double (nullabl

In [5]:
print("dow_johnes: ")
dow_johnes.describe().show()
print("ftse_100: ")
ftse_100.describe().show()
print("nikkei_225: ")
nikkei_225.describe().show()
print("sp_500: ")
sp_500.describe().show()
print("EGX_30: ")
EGX_30.describe().show()

dow_johnes: 
+-------+----------+------------------+-----------------+------------------+------------------+--------------------+
|summary|      Date|             Close|             High|               Low|              Open|              Volume|
+-------+----------+------------------+-----------------+------------------+------------------+--------------------+
|  count|      3522|              3522|             3522|              3522|              3522|                3522|
|   mean|      NULL|21712.507973596767|21825.74421354832|21584.509138983532|21707.796203439986| 2.384138358886996E8|
| stddev|      NULL| 8037.528384265559|8081.914849522401| 7991.416107733758| 8037.466348928835|1.3382979847076967E8|
|    min|2010-01-04|     9686.48046875|  9770.8701171875|      9614.3203125|     9686.48046875|           8410000.0|
|    max|2023-12-29|     37710.1015625|    37778.8515625|    37650.98046875|    37701.62890625|            9.1599E8|
+-------+----------+------------------+------------

In [18]:
dow_johnes = rename_columns(dow_johnes,Indices.dow_Jones.name)
ftse_100 = rename_columns(ftse_100,Indices.ftse_100.name)
nikkei_225 = rename_columns(nikkei_225,Indices.nikkei_225.name)
sp_500 = rename_columns(sp_500,Indices.sp_500.name)
EGX_30 = rename_columns(EGX_30,Indices.EGX_30.name)

In [28]:
merged_df_outer_join = EGX_30.join(sp_500, "Date", "outer") \
                     .join(ftse_100, "Date", "outer") \
                     .join(nikkei_225, "Date", "outer") \
                     .join(dow_johnes, "Date", "outer") 

merged_df_outer_join.count()
merged_df_outer_join = cast_columns_to_double(merged_df_outer_join)

In [29]:
merged_df_outer_join.printSchema()

root
 |-- Date: string (nullable = true)
 |-- EGX_30_Close: double (nullable = true)
 |-- EGX_30_Open: double (nullable = true)
 |-- EGX_30_High: double (nullable = true)
 |-- EGX_30_Low: double (nullable = true)
 |-- EGX_30_Volume: double (nullable = true)
 |-- sp_500_Close: double (nullable = true)
 |-- sp_500_High: double (nullable = true)
 |-- sp_500_Low: double (nullable = true)
 |-- sp_500_Open: double (nullable = true)
 |-- sp_500_Volume: double (nullable = true)
 |-- ftse_100_Close: double (nullable = true)
 |-- ftse_100_High: double (nullable = true)
 |-- ftse_100_Low: double (nullable = true)
 |-- ftse_100_Open: double (nullable = true)
 |-- ftse_100_Volume: double (nullable = true)
 |-- nikkei_225_Close: double (nullable = true)
 |-- nikkei_225_High: double (nullable = true)
 |-- nikkei_225_Low: double (nullable = true)
 |-- nikkei_225_Open: double (nullable = true)
 |-- nikkei_225_Volume: double (nullable = true)
 |-- dow_Jones_Close: double (nullable = true)
 |-- dow_Jones

In [32]:
merged_df_inner_join = EGX_30.join(sp_500, "Date", "inner") \
                     .join(ftse_100, "Date", "inner") \
                     .join(nikkei_225, "Date", "inner") \
                     .join(dow_johnes, "Date", "inner")
merged_df_inner_join.count()
merged_df_inner_join = cast_columns_to_double(merged_df_inner_join)

In [34]:
from pyspark.sql.window import Window  # Import Window for defining window specifications
from pyspark.sql.functions import lag, col, when

# Define a window specification
window_spec = Window.partitionBy().orderBy("Date")

# Calculate daily returns for EGX_30
EGX_30_df = EGX_30.withColumn(
    "daily_return",
    when(
        lag("EGX_30_Close", 1).over(window_spec).isNotNull(),
        (col("EGX_30_Close") - lag("EGX_30_Close", 1).over(window_spec)) / lag("EGX_30_Close", 1).over(window_spec)
    ).otherwise(0)  # Replace null values with 0 or another default
)

In [36]:
EGX_30_df.show()

+----------+------------+-----------+-----------+----------+-------------------+--------------------+
|      Date|EGX_30_Close|EGX_30_Open|EGX_30_High|EGX_30_Low|      EGX_30_Volume|        daily_return|
+----------+------------+-----------+-----------+----------+-------------------+--------------------+
|2010-01-04|      6323.9|    6274.82|    6333.31|   6272.61|            7.057E7|                 0.0|
|2010-01-05|     6420.49|    6350.43|    6430.47|   6349.13|             5.75E7|0.015273802558547756|
|2010-01-06|     6436.87|    6411.13|    6451.92|   6407.23|            8.615E7|0.002551207150856...|
|2010-01-10|     6476.19|    6467.42|    6510.81|   6467.42|             6.99E7|0.006108558973538336|
|2010-01-11|     6459.38|    6454.38|    6463.05|   6436.54|           1.0063E8|-0.00259566195556...|
|2010-01-12|      6534.8|    6460.85|     6543.8|   6460.85|            7.383E7|0.011676043211577593|
|2010-01-13|     6581.59|    6569.36|    6613.62|   6551.04|            8.591E7|0.

In [38]:
correlation = merged_df_inner_join.stat.corr("sp_500_Close", "EGX_30_Close")
print(f"Correlation between SP500 and EGX30: {correlation}")

Correlation between SP500 and EGX30: 0.7177698993251508
